# Homework 4

**Name:** -- Roberto José González --

**e-mail:** -- roberto.jose0745@alumnos.udg.mx --

# MODULES

In [4]:
# ------------------------------
# Load modules and configure environment
# ------------------------------
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from scipy.stats import cauchy, pareto
import panel as pn



# Enable Panel extension for Plotly integration
pn.extension('plotly')

# Set a seed for reproducibility
np.random.seed(42)
print("Environment configured and modules loaded")

Environment configured and modules loaded


Minimum requirements:
* Functions that return the different types of trajectories: Brownian Motion (BM),
Correlated Random Walk (CRW), and Lévy Flight (LF).
* All functions must take as arguments the number of steps, speed, and starting
position.

  * In addition, CRW and LF functions must also take as an argument the Cauchy
coefficient.
  * Lastly, the LF function must also include the Lévy exponent (alpha) as an
argument.

* Functions that compute the metrics: Path length (PL), Mean Squared Displacement
(MSD), and Turning Angle Distribution (TAD).

The dashboard should include at least the following functionalities:
* A panel to display the current trajectory (3D projection).
* A panel to display the graphic for the metric of choice.
* Radio Items (or a similar type of selector) to select the type of trajectory (BM, CRW, LF).
* Drop down menu (or similar) to select the metric to be displayed (PL, MSD, TAD).
* Widgets to the trajectories’ parameters (integer and floating point values). These widgets
must be dynamically displayed according to the type of trajectory selected.

In [5]:
# Trajectory generation functions
def generate_BM(steps, speed, start):
    """
    Generate a 3D Brownian Motion (BM) trajectory.
    Each step is a random normally distributed displacement scaled by the speed.
    """
    traj = [np.array(start)]
    for i in range(steps):
        step = np.random.normal(scale=speed, size=3)
        traj.append(traj[-1] + step)
    return np.array(traj)

def generate_CRW(steps, speed, start, cauchy_coef):
    """
    Generate a 3D Correlated Random Walk (CRW) trajectory.
    Uses a previous direction and adds a perturbation sampled from a Cauchy distribution.
    """
    traj = [np.array(start)]
    # Initialize a normalized random direction
    direction = np.random.randn(3)
    direction /= np.linalg.norm(direction)
    for i in range(steps):
        # Generate a perturbation from the Cauchy distribution
        perturbation = cauchy.rvs(scale=cauchy_coef, size=3)
        direction = direction + perturbation
        direction /= np.linalg.norm(direction)
        step = direction * speed
        traj.append(traj[-1] + step)
    return np.array(traj)

def generate_LF(steps, speed, start, cauchy_coef, alpha):
    """
    Generate a 3D Lévy Flight (LF) trajectory.
    The step lengths follow a heavy-tailed Pareto distribution (shifted) and the direction is perturbed with Cauchy noise.
    """
    traj = [np.array(start)]
    for i in range(steps):
        # Sample a step length from a Pareto distribution (shifted by +1) and scaled by speed
        step_length = (pareto.rvs(alpha) + 1) * speed
        direction = np.random.randn(3)
        direction /= np.linalg.norm(direction)
        perturbation = cauchy.rvs(scale=cauchy_coef, size=3)
        direction = direction + perturbation
        direction /= np.linalg.norm(direction)
        step = direction * step_length
        traj.append(traj[-1] + step)
    return np.array(traj)

# Metric calculation functions

def metric_PL(traj):
    """
    Calculate the Path Length (PL) of the trajectory.
    Sum the Euclidean distances between consecutive points.
    """
    distances = np.linalg.norm(np.diff(traj, axis=0), axis=1)
    return np.sum(distances)

def metric_MSD(traj):
    """
    Calculate the Mean Squared Displacement (MSD) of the trajectory.
    Computes the average squared distance from the starting point.
    """
    displacements = traj - traj[0]
    squared_distances = np.sum(displacements**2, axis=1)
    return np.mean(squared_distances)

def metric_TAD(traj):
    """
    Calculate the Turning Angle Distribution (TAD) of the trajectory.
    Computes the angles (in radians) between consecutive steps.
    """
    steps = np.diff(traj, axis=0)
    angles = []
    for i in range(1, len(steps)):
        dot_product = np.dot(steps[i-1], steps[i])
        norm_product = np.linalg.norm(steps[i-1]) * np.linalg.norm(steps[i])
        if norm_product == 0:
            angle = 0
        else:
            cos_angle = np.clip(dot_product / norm_product, -1, 1)
            angle = np.arccos(cos_angle)
        angles.append(angle)
    return np.array(angles)


In [ ]:
# Dashboard widgets and plot setup using panel and plotly

# Create widgets for user inputs
trajectory_selector = pn.widgets.RadioButtonGroup(
    name='Trajectory Type', options=['BM', 'CRW', 'LF'], value='BM'
)
metric_selector = pn.widgets.Select(
    name='Metric', options=['PL', 'MSD', 'TAD'], value='PL'
)
steps_widget = pn.widgets.IntInput(name='Number of Steps', value=100)
speed_widget = pn.widgets.FloatInput(name='Speed', value=1.0)
start_x_widget = pn.widgets.FloatInput(name='Start X', value=0.0)
start_y_widget = pn.widgets.FloatInput(name='Start Y', value=0.0)
start_z_widget = pn.widgets.FloatInput(name='Start Z', value=0.0)
cauchy_coef_widget = pn.widgets.FloatInput(name='Cauchy Coefficient', value=1.0)
alpha_widget = pn.widgets.FloatInput(name='Lévy Exponent (alpha)', value=1.5)

# Set initial visibility for extra parameters (only used for CRW and LF)
cauchy_coef_widget.visible = False
alpha_widget.visible = False

# Callback Functions for Interactivity

def update_dashboard(event=None):
    """
    Main update function:
    - Generates the selected trajectory based on widget parameters.
    - Computes the chosen metric.
    - Updates the Plotly figures for the 3D trajectory and the metric.
    """
    # Retrieve widget values
    traj_type = trajectory_selector.value
    steps = steps_widget.value
    speed = speed_widget.value
    start = [start_x_widget.value, start_y_widget.value, start_z_widget.value]
    
    # Retrieve additional parameters if visible
    cauchy_coef = cauchy_coef_widget.value if cauchy_coef_widget.visible else 0
    alpha = alpha_widget.value if alpha_widget.visible else 0
    
    # Generate trajectory according to the selected type
    if traj_type == 'BM':
        traj = generate_BM(steps, speed, start)
    elif traj_type == 'CRW':
        traj = generate_CRW(steps, speed, start, cauchy_coef)
    elif traj_type == 'LF':
        traj = generate_LF(steps, speed, start, cauchy_coef, alpha)
    else:
        traj = generate_BM(steps, speed, start)
    
    # Calculate selected metric
    metric_choice = metric_selector.value
    if metric_choice == 'PL':
        metric_value = metric_PL(traj)
    elif metric_choice == 'MSD':
        metric_value = metric_MSD(traj)
    elif metric_choice == 'TAD':
        metric_value = metric_TAD(traj)
    else:
        metric_value = None
    
    # Create a 3D trajectory plot using Plotly
    traj_fig = go.Figure(data=[go.Scatter3d(
        x=traj[:,0],
        y=traj[:,1],
        z=traj[:,2],
        mode='lines+markers',
        line=dict(width=2),
        marker=dict(size=3)
    )])
    traj_fig.update_layout(title=f"{traj_type} Trajectory (3D)",
                           scene=dict(
                               xaxis_title="X",
                               yaxis_title="Y",
                               zaxis_title="Z"
                           ))
    
    # Create the metric plot
    if metric_choice == 'TAD' and metric_value is not None:
        # For TAD, plot a histogram of turning angles
        metric_fig = px.histogram(metric_value, nbins=20,
                                  title="Turning Angle Distribution (Histogram)")
    else:
        # For PL and MSD, display the metric value as a text annotation in a blank figure
        metric_fig = go.Figure()
        metric_fig.add_annotation(
            text=f"{metric_choice} Value: {metric_value:.2f}",
            x=0.5, y=0.5, showarrow=False,
            font=dict(size=20)
        )
        metric_fig.update_layout(title=f"{metric_choice} Value",
                                 xaxis={'visible': False}, yaxis={'visible': False})
    
    # Update Panel panes with the new figures
    traj_pane.object = traj_fig
    metric_pane.object = metric_fig

def update_widgets(event):
    """
    Update the visibility of extra parameter widgets based on the selected trajectory type:
    - BM: Hide cauchy_coef and alpha.
    - CRW: Show cauchy_coef only.
    - LF: Show both cauchy_coef and alpha.
    """
    if trajectory_selector.value == 'BM':
        cauchy_coef_widget.visible = False
        alpha_widget.visible = False
    elif trajectory_selector.value == 'CRW':
        cauchy_coef_widget.visible = True
        alpha_widget.visible = False
    elif trajectory_selector.value == 'LF':
        cauchy_coef_widget.visible = True
        alpha_widget.visible = True
    update_dashboard()

# Create panel panes to hold plotly figures
traj_pane = pn.pane.Plotly(height=400, width=500)
metric_pane = pn.pane.Plotly(height=400, width=500)

# Set up Watchers for Interactive Updates
trajectory_selector.param.watch(update_widgets, 'value')
steps_widget.param.watch(lambda event: update_dashboard(), 'value')
speed_widget.param.watch(lambda event: update_dashboard(), 'value')
start_x_widget.param.watch(lambda event: update_dashboard(), 'value')
start_y_widget.param.watch(lambda event: update_dashboard(), 'value')
start_z_widget.param.watch(lambda event: update_dashboard(), 'value')
cauchy_coef_widget.param.watch(lambda event: update_dashboard(), 'value')
alpha_widget.param.watch(lambda event: update_dashboard(), 'value')
metric_selector.param.watch(lambda event: update_dashboard(), 'value')

# Call update_dashboard initially to render the figures
update_dashboard()

# Layout the dashboard using panel
widgets_column = pn.Column(
    trajectory_selector,
    steps_widget,
    speed_widget,
    start_x_widget,
    start_y_widget,
    start_z_widget,
    cauchy_coef_widget,
    alpha_widget,
    metric_selector
)

plots_row = pn.Row(traj_pane, metric_pane)
dashboard = pn.Row(widgets_column, plots_row)


BokehModel(combine_events=True, render_bundle={'docs_json': {'07ce17bb-543e-432f-9476-8b50e55f47a4': {'version…

In [ ]:
# Display the dashboard in the notebook
dashboard